In [32]:
import gurobipy as gp
import numpy as np
import pandas as pd

Read data

In [33]:
import re

def parse_dzn(file_path):
    data = {}
    with open(file_path, 'r') as file:
        content = file.read()
        
        # Find all key-value pairs
        key_value_pairs = re.findall(r'(\w+)\s*=\s*(.*?);', content, re.DOTALL)
        
        for key, value in key_value_pairs:
            # Remove any extra whitespace and newlines
            value = value.strip().replace('\n', '')

            #Parse matrix
            if key == 'D':
                rows = value.split('|')
                D = []
                for i in range(1,len(rows)-1):
                    D.append([int(v.strip()) for v in rows[i][1:-1].split(',')])
                value = D
            # Parse arrays
            elif value.startswith('[') and value.endswith(']'):
                value = value[1:-1].split(',')
                value = [int(v.strip()) for v in value]
            else:
                value = int(value)
                
            data[key] = value
    return data

# Usage
'''
file_path = 'inst08.dzn'  # Path to your .dzn file
parsed_data = parse_dzn(file_path)
print(parsed_data)
'''

"\nfile_path = 'inst08.dzn'  # Path to your .dzn file\nparsed_data = parse_dzn(file_path)\nprint(parsed_data)\n"

Create the model

In [34]:
model = gp.Model()

Parse the instance

In [35]:
file_path = input('Type instance name:\n>')
parsed_data = parse_dzn(file_path)

m = parsed_data['m']
n = parsed_data['n']
s = parsed_data['s']
l = parsed_data['l']
D = parsed_data['D']

Define the variables

In [36]:
#distances = model.addVars(m,n+1,lb=0, vtype=gp.GRB.INTEGER, name='Distances')

max_total_dist = model.addVar(vtype=gp.GRB.CONTINUOUS, name='MaxTotalDist')
#max_total_dist_for_c = model.addVars(m, vtype=gp.GRB.INTEGER, name='MaxTotalDistForC')
#max_load_for_c = model.addVars(m, vtype=gp.GRB.INTEGER, name='MaxLoadForC')

cumulative_dist = model.addVars(m, n+1, vtype=gp.GRB.INTEGER, name='CumulativeDist')
cumulative_load = model.addVars(m, n+1, lb=0, ub=n*max(s), vtype=gp.GRB.INTEGER, name='CumulativeLoad')

paths = model.addVars(m, n+1, n+1, vtype=gp.GRB.BINARY, name='Paths')

Define Objective function

In [37]:
model.setObjective(max_total_dist, gp.GRB.MINIMIZE)

Define Constraints

In [38]:
#Maximum total distance is equal to the maximum total distance computed in the final depot "slot" for each courier
model.addConstrs( gp.quicksum(cumulative_dist[c,j] for j in range(n+1)) <= max_total_dist for c in range(m))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [39]:
#Every node must be visited (except for the depot n+1)
for dest in range(n):
    # model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) <= 1.5)
    # model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) >= 0.5)
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) == 1)

In [40]:
#Every courier must visit the depot as last node
for c in range(m):
    # model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n+1)) >= 0.5)
    # model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n+1)) <= 1.5)
    model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n+1)) == 1)

In [41]:
#Couriers cannot stay still
for c in range(m):
    for source in range(n+1):
        model.addConstr(paths[c,source,source] <= 0.5)

In [42]:
#Each courier must start from depot
for c in range(m):
    model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n+1)) >= 0.5)

In [43]:
for c in range(m):
    for source in range(n+1):
        for dest in range(n+1):
            #Cumulative distance update
            model.addConstr((paths[c,source,dest] == 1) >> (cumulative_dist[c,dest] == D[source][dest]))

            #Path contiguity
            model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,dest,j] for j in range(n+1)) >= 0.5))

            #Load update
            if dest < n:
                model.addConstr((paths[c,source,dest] == 1) >> (cumulative_load[c,dest] == s[dest]))

In [44]:
#Loop avoidance
# for c in range(m):
#     for source in range(n):
#         for dest in range(n):
#             if source < dest:
#                 model.addConstr( (paths[c,source,dest] == 1) >> (paths[c,dest,source] <= 0.5) )

In [45]:
#Maximum load
for c in range(m):
    model.addConstr(gp.quicksum(cumulative_load[c,j] for j in range(n+1)) <= l[c])

In [46]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 127 columns and 156 nonzeros
Model fingerprint: 0x14bd4f3e
Model has 280 general constraints
Variable types: 1 continuous, 126 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+01]
  RHS range        [5e-01, 2e+01]
  GenCon rhs range [5e-01, 8e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 140 rows and 54 columns
Presolve time: 0.01s
Presolved: 168 rows, 181 columns, 852 nonzeros
Presolved model has 84 SOS constraint(s)
Variable types: 0 continuous, 181 integer (96 binary)
Found heuristic solution: objective 22.0000000
Found heuristic solution: objective 18.0000000

Root relaxation: objective 0.000000e+00, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current 

In [47]:
for c in range(m):
    print(c)
    rows = [i for i in range(n+1)]
    adj_matrix = pd.DataFrame(columns=rows, index=rows)
    for i in rows:
        for j in rows:
            adj_matrix.loc[i,j] = int(paths[c,i,j].X)
    print(adj_matrix)

0
   0  1  2  3  4  5  6
0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0
2  0  0  0  0  0  0  1
3  0  0  0  0  0  0  0
4  0  0  0  0  0  1  0
5  0  0  0  0  1  0  0
6  0  0  1  0  0  0  0
1
   0  1  2  3  4  5  6
0  0  1  0  0  0  0  0
1  1  0  0  0  0  0  0
2  0  0  0  0  0  0  0
3  0  0  0  0  0  0  1
4  0  0  0  0  0  0  0
5  0  0  0  0  0  0  0
6  0  0  0  1  0  0  0
